In [ ]:
"""
The intent behind this notebook was to calculate additional features that'd be a pain to do in tableau.
Currently this is just the win-rates and pick-rates.

"""


In [1]:

import pandas as pd
import time
#data=req.json()
import matplotlib.pyplot as plt
import os
import numpy as np
from datetime import datetime

In [121]:
#move this and other paramters to separate file
support_heroes={'Ana','Baptiste','Brigette','Lucio','Mercy','Moira','Zenyatta'}
tank_heroes={'D.Va','Orisa','Reinhart','Roadhog','Sigma','Winston','Wrecking Ball','Zarya'}
damage_heroes={'Ashe',
 'Bastion',
 'Doomfist',
 'Echo',
 'Genji',
 'Hanzo',
 'Junkrat',
 'McGree',
 'Mei',
 'Pharah',
 'Reaper',
 'Soldier: 76',
 'Sombra',
 'Symmetra',
 'Torbjorn',
 'Tracer',
 'Widowmaker'}

In [122]:
log_path='C:\\Users\\Matt\\OneDrive\\'
custom_date_parser = lambda x: datetime.strptime(x, "%Y_%m_%d")
pivoted_ow_df=pd.read_csv(log_path+'full_comp_stats_running.csv', 
                          parse_dates=['date'],
                          index_col=('player','hero','date'),
                          date_parser=custom_date_parser)
# pivoted_ow_df=pd.read_csv(log_path+'full_comp_stats_running.csv', 

#                           index_col=('player','hero','date'),
# )
pivoted_ow_df.sort_index(inplace=True)
#1/11/2021, why did I reset the index. Could/should have just operated on it that form.
new_df=pivoted_ow_df.reset_index()
#new_df['date']=pd.to_datetime(new_df['date'],format='%Y_%m_%d')

In [123]:
#run this for squad, then start looking at 3 cases- when all role losses or wins in day, or when equal.
cols=['Time Played',
'Games Lost',
'Games Tied',
'Games Won',
'Games Played']
roles=['Damage','Tank','Support']
class OW_player_analysis():
    def __init__(self,df):
        #df is full pivoted table for one player
        self.df=df
        self.all_heroes=df[df['hero']=='All Heroes'].set_index('date').sort_index()#.dropna(axis=1,how='all')
        self.sr_diff=self.all_heroes[['Tank SR','Support SR','Damage SR','Games Played']].diff()
    def specific_hero_diff(self,hero):
        hero_df=self.df[self.df['hero']==hero].set_index('date').sort_index().dropna(axis=1,how='all').drop(['player','hero'],axis=1)
        hero_diff_df=hero_df.diff()
        #Consider adding code here to fix the discontinuity between seasons
        return hero_df,hero_diff_df
    def check_role_changes(self):
        """
        What are we doing here? 
        What do we want? Support/tank/dps games played/won/lost/tied/time, along with SR changes
        
        Net positive avg change, net negative avg change, net neutral avg change
        
        Looks like a +29 for a net 0 on support for me on 5/15, which is not feasible. But this is not an error
        in my code, that's how it is in the html files.
        Also Net - Tank SR seems too low for 5/15, consider dropping that day
        
        Berg's 4/29 also makes no sense, a -29 despite having a new win. Dropped game? 
        Add sanity checks, if net 0 change>15, net pos<10, net negative>-10
        """
        self.support=self.df.loc[new_df['hero'].isin(support_heroes),cols+['date']].groupby('date').sum().add_suffix(' Support')
        self.support_dff=self.support.sort_index().diff()
        
        self.damage=self.df.loc[new_df['hero'].isin(damage_heroes),cols+['date']].groupby('date').sum().add_suffix(' Damage')
        self.damage_dff=self.damage.sort_index().diff()
        
        self.tank=self.df.loc[new_df['hero'].isin(tank_heroes),cols+['date']].groupby('date').sum().add_suffix(' Tank')
        self.tank_dff=self.tank.sort_index().diff()
        
        
        self.all_roles=self.support.join(self.damage).join(self.tank,rsuffix='tank')
        self.all_roles_dff=self.all_roles.sort_index().diff()
        self.all_roles_dff_sr=self.all_roles_dff.join(self.sr_diff)
        for role in roles:
            self.all_roles_dff_sr[f"Net {role} Games"]=self.all_roles_dff_sr[f"Games Won {role}"]-self.all_roles_dff_sr[f"Games Lost {role}"]
            net_pos_mask=((self.all_roles_dff_sr[f"Net {role} Games"]>0) &(self.all_roles_dff_sr[f"Games Won {role}"]>0) )
            net_neg_mask=((self.all_roles_dff_sr[f"Net {role} Games"]<0) &(self.all_roles_dff_sr[f"Games Lost {role}"]>0) )
            net_zero_mask=((self.all_roles_dff_sr[f"Net {role} Games"]==0) &(self.all_roles_dff_sr[f"Games Won {role}"]>0) )
            self.all_roles_dff_sr.loc[net_pos_mask,f'Net + SR {role}']=self.all_roles_dff_sr.loc[net_pos_mask,f"{role} SR"]/self.all_roles_dff_sr.loc[net_pos_mask,f"Net {role} Games"]
            self.all_roles_dff_sr.loc[net_neg_mask,f'Net - SR {role}']=self.all_roles_dff_sr.loc[net_neg_mask,f"{role} SR"]/self.all_roles_dff_sr.loc[net_neg_mask,f"Net {role} Games"]
            self.all_roles_dff_sr.loc[net_zero_mask,f'Net 0 SR {role}']=self.all_roles_dff_sr.loc[net_zero_mask,f"{role} SR"]/self.all_roles_dff_sr.loc[net_zero_mask,f"Games Won {role}"]
            #Sanity checks. SR change <12, >40 unreasonable? 
            self.all_roles_dff_sr.loc[self.all_roles_dff_sr[f'Net + SR {role}']>40,f'Net + SR {role}']=np.nan
            self.all_roles_dff_sr.loc[self.all_roles_dff_sr[f'Net + SR {role}']<12,f'Net + SR {role}']=np.nan
            self.all_roles_dff_sr.loc[self.all_roles_dff_sr[f'Net - SR {role}']>40,f'Net - SR {role}']=np.nan
            self.all_roles_dff_sr.loc[self.all_roles_dff_sr[f'Net - SR {role}']<12,f'Net - SR {role}']=np.nan
            self.all_roles_dff_sr.loc[self.all_roles_dff_sr[f'Net 0 SR {role}'].abs()>15,f'Net 0 SR {role}']=np.nan

In [ ]:
#So what do we want to do?
"""
Per player/role, show how win percentages change over time for various heroes within that role. 
While you could simply use the win-percentage as is, that's tied to the season. Could instead do it as a rolling average of X
number of games played with that hero, say 10.
"""

In [124]:
#okay, this seems to work. Consider making the window size a parameter
def get_rolling_wr(m_df_slim,pivoted_ow_df,hero,player):
    played_list=[]
    won_list=[]
    min_window_size=25
    max_window_size=40
    out_series=pd.Series(dtype='float64')
    m_df_slim.fillna(value=0,inplace=True)
    for index, row in m_df_slim.iterrows():
        if row['Games Played']>=0:
            played_list.append(row['Games Played'])
            won_list.append(row['Games Won'])
        if sum(played_list)>min_window_size:
            while sum(played_list)>max_window_size:
                #Perhaps add a check to make sure removing the next one doesn't drop below the min threshold
                played_list=played_list[1:]
                won_list=won_list[1:]
            win_rate=sum(won_list)/sum(played_list)*100
            out_series.loc[index]=win_rate
            
#Kinda surprised I never hit any index errors with this. Makes sense though, it's operating off of the character
#specific series.
    pivoted_ow_df.loc[(player,hero,slice(out_series.index[0],out_series.index[-1])),'Win Rate']=out_series.values
    return out_series

In [125]:
def get_pick_rates(pr_df,pivoted_ow_df,player,missing_index_dict):
    played_list=[]
    index_list=[]
    min_window_size=25
    max_window_size=40
    hero_cols=[col for col in pr_df.columns if col!=f'{role} played' ]
    out_df=pd.DataFrame()
    index_dict={}
    pr_dict={}
    
    for hero in hero_cols:
        pr_dict[hero]=[]
        index_dict[hero]=[]
    for index, row in pr_df.iterrows():
        if row[f'{role} played']>=0:
            played_list.append(row[f'{role} played'])
            index_list.append(index)
        if sum(played_list)>min_window_size:
            while sum(played_list)>max_window_size:
                #Perhaps add a check to make sure removing the next one doesn't drop below the min threshold
                played_list=played_list[1:]
                index_list=index_list[1:]
            for hero in hero_cols:
                pr=sum(pr_df.loc[index_list[0]:index_list[-1],hero])/sum(played_list)*100
                #series_dict[hero].loc[index]=pr
                #pivoted_ow_df.loc[(player,hero,index),'Pick Rate']=pr
                if index in pivoted_ow_df.loc[(player,hero)].index:pivoted_ow_df.loc[(player,hero,index),'Pick Rate']=pr
                else:missing_index_dict[(player,hero,index)]=pr
                    
                    
    """
    1/13/2021 5pm
    So apparently the reason for the slow performance, index unsorted error was caused by try to insert data into a 
    date for which that particular hero had no info, it's creating a new row. Simply checking to see if the index existed fixes
    this. Now won't this cause issues, because of how the rolling window works between seasons, you'll have a non-zero pick rate
    be omitted from the data. Let's see when we work with it in Tableau. 
    An easy fix might to up at the top go through and add all the missing day/hero/combinations, which would increase the size of the
    DF, but not too large. 
    Never did chase down why the slice method was failing for my Junkrat data, but it seems moot now. Let's table this for now
    and work on Tableau.
    
    """                    
                    

    return None

In [126]:
squad=['Rthunder27','BIGoleICEBERG','Seraph341','LaCroixDaddy','YAS RIHANNA','star4ker','ULove2SeeIt7915']
missing_index_dict=dict()
#player='Rthunder27'
for player in squad:
    t0=time.time()
    t=OW_player_analysis(new_df.loc[new_df['player']==player])
    role_dict={'Tank':tank_heroes,'Support':support_heroes,'DPS':damage_heroes}
    for role in role_dict:
        pivoted_ow_df.sort_index(inplace=True)
        pr_df=pd.DataFrame()
        wr_df=pd.DataFrame()
        for hero in role_dict[role]:
            try:
                m,m_df=t.specific_hero_diff(hero)
            except:continue
                #Yes, this means that the first games played with a hero each season will be ignored. need better way to fix season change discontinuities
            pr_df[hero]=m_df['Games Played'].apply(lambda x: 0 if x<0 else x)

            try:
                wr_series=get_rolling_wr(m_df,pivoted_ow_df,hero,player)

            except:continue
        pr_df.fillna(value=0,inplace=True)
        pr_df[f'{role} played']=pr_df.sum(axis=1)   
        get_pick_rates(pr_df,pivoted_ow_df,player,missing_index_dict)

    print((time.time()-t0))
missing_indices_df=pd.DataFrame(index=missing_index_dict.keys(),data=missing_index_dict.values(),columns=['Pick Rate'])
ow_df_w_missings=pd.concat([pivoted_ow_df,missing_indices_df])    

4.332566022872925
5.710983991622925
3.1319241523742676
3.5163800716400146
6.447938680648804
0.6447956562042236
1.1386241912841797


In [112]:
#1/26 Add in logic to fill missings in SR

In [127]:
ow_df_w_missings.to_csv('full_comp_with_rates_2021-01-25.csv')